In [ ]:
from google.colab import drive
drive.mount("/content/drive")
project_dir = "/content/drive/MyDrive/streamingllm"


Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/streamingllm

/content/drive/MyDrive/streamingllm


In [ ]:
!python setup.py develop

running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        ********************************************************************************

!!
  easy_install.initialize_options(self)
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.



In [ ]:
!pip install torch torchvision torchaudio
!pip install transformers==4.33.0 accelerate datasets evaluate wandb scikit-learn scipy sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.0/254.0 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: 

In [ ]:
!pip install langchain
!pip install sentence_transformers
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=996bbfb10a46c49f08479ebe5c01861a6b1e5ba7614e13bb55011c4225e74575
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.0/506.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.9 MB/

In [ ]:
import torch
from tqdm import tqdm
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, RagRetriever, RagSequenceForGeneration
from datasets import load_dataset
from torch.nn import CrossEntropyLoss
from streaming_llm.kv_cache import StartRecentKVCache
from streaming_llm.utils import parse_args, load

### ADDED #####
import numpy as np
from langchain.chains import ConversationalRetrievalChain
from langchain.retrievers import WikipediaRetriever
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
################

device = "cuda"

#### args ####
dataset_name = "wikitext"
task = "wikitext-2-raw-v1"
split = "test"
model_name_or_path = "lmsys/vicuna-7b-v1.5"
enable_start_recent_kv_cache = True
enable_pos_shift = True
start_size = 4
recent_size = 256
num_samples = 50
output_dir = "outputs/debug_wo_rag"
arg_num_eval_tokens = None
start_ret = 3
###############

##### helper #####
def find_first(arr, seq):
    arr_with_indices = torch.cat([arr.reshape(-1, 1),
                         torch.Tensor([i for i in range(arr.size(0))]).reshape(-1, 1)],
                                 dim=1)
    for i in range(arr.size(0)):
        if torch.equal(arr_with_indices[:, 0][arr != 1][i:i+seq.size(0)], seq):
            return (arr_with_indices[:,1][arr != 1][i].int(),
                    arr_with_indices[:,1][arr != 1][i + seq.size(0) - 1].int())
    return None
################
model_name = "all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=20, chunk_overlap=3)
#################



data = load_dataset(dataset_name, task, split=split)
model, tokenizer = load(model_name_or_path)



nlls = []
loss_fn = CrossEntropyLoss(reduction="none")
past_key_values = None

if enable_start_recent_kv_cache:
    if "llama" in model.config.model_type:
        k_seq_dim = v_seq_dim = 2
    elif "mpt" in model.config.model_type:
        v_seq_dim = 2
        k_seq_dim = 3
    elif "pythia" in model.config.model_type:
        k_seq_dim = v_seq_dim = 2
    elif "falcon" in model.config.model_type:
        v_seq_dim = 1
        k_seq_dim = 1
    else:
        raise ValueError(f"got {model.config.model_type}")
    kv_cache = StartRecentKVCache(
        start_size=start_size,
        recent_size=recent_size,
        k_seq_dim=k_seq_dim,
        v_seq_dim=v_seq_dim,
    )
else:
    kv_cache = None

if enable_pos_shift:
    if "llama" in model.config.model_type:
        from streaming_llm.pos_shift.modify_llama import enable_llama_pos_shift_attention

        enable_llama_pos_shift_attention(model)
    elif "falcon" in model.config.model_type:
        from streaming_llm.pos_shift.modify_falcon import (
            enable_falcon_pos_shift_attention,
        )

        enable_falcon_pos_shift_attention(model)
    elif "gpt_neox" in model.config.model_type:
        from streaming_llm.pos_shift.modify_gpt_neox import (
            enable_gpt_neox_pos_shift_attention,
        )

        enable_gpt_neox_pos_shift_attention(model)
    elif "mpt" in model.config.model_type:
        pass
    else:
        raise ValueError(f"got {model.config.model_type}")


os.makedirs(output_dir, exist_ok=True)
f = open(f"{output_dir}/log.txt", "w")
ppls = []
for k in range(0, 10*num_samples, num_samples):
    print(f"trial: {k}")
    num_eval_tokens = 0
    i = 0
    encs = None
    for text in data["text"][k : k + num_samples]:
        encodings = tokenizer(text, return_tensors="pt")

        #### ADDED ######
        if encs is None:
            encs = encodings.input_ids.clone().detach().cpu()
        else:
            encs = torch.cat([encs, encodings.input_ids.detach().cpu()], dim = 1)
        keeps = None
        if i > start_ret:
            tsofar = " ".join(data['text'][k:k+i])
            texts = text_splitter.split_text(tsofar)
            db = Chroma.from_texts(texts, embedding)
            retriever = db.as_retriever(search_kwargs={'k': 10})
            retrieved_docs = retriever.invoke(text)
            tokens = [tokenizer(doc.page_content, return_tensors="pt").input_ids for
                    doc in retrieved_docs]
            steds = [find_first(encs.flatten(), token.flatten()[1:])
                    for token in tokens]
            steds = [s for s in steds if s is not None]
            if steds:
                keeps = sum([ed - st + 1 for st, ed in steds]), steds
            else:
                print("No retrievel")
        ################


        print(encodings.input_ids[:, :10])

        seq_len = encodings.input_ids.size(1)
        print(f"seq_len: {seq_len}")
        pbar = tqdm(range(0, seq_len - 1))

        for idx in pbar:
            input_ids = encodings.input_ids[:, idx : idx + 1].to(device)
            with torch.no_grad():
                outputs = model(
                    input_ids,
                    past_key_values=past_key_values,
                    use_cache=True,
                )
                logits = outputs.logits.view(-1, model.config.vocab_size).to('cpu')
                past_key_values = outputs.past_key_values
                label = encodings.input_ids[:, idx + 1 : idx + 2].to(logits.device).view(-1)
                neg_log_likelihood = loss_fn(logits, label)
                if kv_cache is not None:
                    # changed kv_cache with keeps values to keep attention for retrieved indices
                    past_key_values = kv_cache(past_key_values, keeps=None)
                    torch.cuda.empty_cache()
            nlls.append(neg_log_likelihood)
            pbar.set_description(
                f"nll: {neg_log_likelihood.item():.2f}, ppl: {torch.exp(neg_log_likelihood).item():.2f}"
            )
            print(neg_log_likelihood.item(), file=f, flush=True)
            num_eval_tokens += 1
            if arg_num_eval_tokens is not None and num_eval_tokens >= arg_num_eval_tokens:
                break
        if arg_num_eval_tokens is not None and num_eval_tokens >= arg_num_eval_tokens:
            break
        i += 1
    f.close()

    ppl = torch.exp(torch.stack(nlls).mean())
    print(ppl.item())
    ppls.append(ppl)
    with open(f"{output_dir}/ppl.txt", "w") as f:
        f.write(f"{ppl.item()}\n")

Loading model from lmsys/vicuna-7b-v1.5 ...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


StartRecentKVCache: 4, 256
trial: 0
tensor([[1]])
seq_len: 1


0it [00:00, ?it/s]


...

tensor([[    1, 29871, 10015, 28944, 29879,   313,   450,   383,  5621,  1723]])
seq_len: 27


100%|██████████| 26/26 [00:01<00:00, 19.09it/s]


tensor([[    1, 29871,   678,   328,  4111,  1416,   359,   313,   678,   466]])
seq_len: 29


100%|██████████| 28/28 [00:01<00:00, 18.69it/s]

9.04964828491211


In [ ]:
import warnings

warnings.filterwarnings("ignore")

import torch
import argparse
import json
import os
import time
import re
import sys

from tqdm import tqdm
from streaming_llm.utils import load, download_url, load_jsonl
from streaming_llm.enable_streaming_llm import enable_streaming_llm

### added ####
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
#############

@torch.no_grad()
def greedy_generate(model, tokenizer, input_ids, past_key_values, max_gen_len, f):
    outputs = model(
        input_ids=input_ids,
        past_key_values=past_key_values,
        use_cache=True,
    )
    past_key_values = outputs.past_key_values
    pred_token_idx = outputs.logits[:, -1, :].argmax(dim=-1).unsqueeze(1)
    generated_ids = [pred_token_idx.item()]
    pos = 0
    for _ in range(max_gen_len - 1):
        outputs = model(
            input_ids=pred_token_idx,
            past_key_values=past_key_values,
            use_cache=True,
        )
        past_key_values = outputs.past_key_values
        pred_token_idx = outputs.logits[:, -1, :].argmax(dim=-1).unsqueeze(1)
        generated_ids.append(pred_token_idx.item())
        generated_text = (
            tokenizer.decode(
                generated_ids,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
                spaces_between_special_tokens=False,
            )
            .strip()
            .split(" ")
        )

        now = len(generated_text) - 1
        if now > pos:
            f.write(" ".join(generated_text[pos:now]) + " ")
            print(" ".join(generated_text[pos:now]), end=" ", flush=True)
            pos = now


        if pred_token_idx == tokenizer.eos_token_id:
            break
    print(" ".join(generated_text[pos:]), flush=True)
    f.write(" ".join(generated_text[pos:]) + " \n")
    return past_key_values


@torch.no_grad()
def streaming_inference(model, tokenizer, prompts, kv_cache=None, max_gen_len=1000):
    past_key_values = None
    f = open("summary_rag_brief.txt", "w")
    #### new ####
    token_st = 0
    token_lens = []
    model_name = "all-MiniLM-L6-v2"
    model_kwargs = {'device': 'cpu'}
    embedding = HuggingFaceEmbeddings(model_name=model_name,
                                    model_kwargs=model_kwargs)
    ###############
    for idx, prompt in enumerate(prompts):
        prompt = "USER: " + prompt + "\n\nASSISTANT: "
        print("\n" + prompt, end="")
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids

        input_ids = input_ids.to(model.device)
        seq_len = input_ids.shape[1]

        #### new ######
        token_lens.append(token_st)
        token_st += seq_len
        keeps = None
        if idx > 5:
            db = Chroma.from_texts(prompts[:idx], embedding)
            retriever = db.as_retriever(search_kwargs={'k': 1})
            retrieved_docs = retriever.invoke(prompt)
            retrieved_idx = prompts.index(retrieved_docs[0].page_content)
            if retrieved_idx != -1:
                keeps  = (
                    seq_len,
                    [(
                        token_lens[retrieved_idx],
                        token_lens[retrieved_idx] + seq_len
                    )]
                )
                print(keeps)

        ###############

        if kv_cache is not None:
            space_needed = seq_len + max_gen_len
            past_key_values = kv_cache.evict_for_space(past_key_values,
                                                       space_needed,
                                                       keeps=None)
            torch.cuda.empty_cache()

        past_key_values = greedy_generate(
            model, tokenizer, input_ids, past_key_values, max_gen_len=max_gen_len, f=f
        )

    f.close()
#### args ####
model_name_or_path = "lmsys/vicuna-7b-v1.5"
enable_streaming = True
start_size = 4
recent_size = 2000
###############

#### prompt #####
with open("long/nfu.txt") as f:
    longprompt = f.read()

# texts = ["\n".join(lines)]#[i:i+7]) for i in range(0, len(lines), 7)]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=40)
texts = text_splitter.split_text(longprompt)
print(f'number of chunks: {len(texts)}')
texts = ["Summarize briefly the following text in English. \n" + text for text in texts]
model, tokenizer = load(model_name_or_path)


prompts = texts
if enable_streaming:
    kv_cache = enable_streaming_llm(
        model, start_size=start_size, recent_size=recent_size
    )
else:
    kv_cache = None

streaming_inference(
    model,
    tokenizer,
    prompts,
    kv_cache,
)

number of chunks: 229
Loading model from lmsys/vicuna-7b-v1.5 ...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

StartRecentKVCache: 4, 2000

USER: Summarize briefly the following text in English. 
﻿The Project Gutenberg eBook of Notes from the Underground
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: Notes from the Underground


Author: Fyodor Dostoyevsky

Release date: July 1, 1996 [eBook #600]
                Most recently updated: December 26, 2021

Language: English



*** START OF THE PROJECT GUTENBERG EBOOK NOTES FROM THE UNDERGROUND ***



Notes from the Underground

by Fyodor Dostoyevsky


Contents

 NOTES FROM THE UNDERGROUND

 PART I Underground
 I
 II
 III
 IV
 V
 VI
 VII
 VIII
 IX


KeyboardInterrupt: ignored

In [ ]:
with open("summary_new_new_map_rag.txt") as f:
    total_count = 0
    for line in f.readlines():
        print(len(line.split()))
        total_count += len(line.split())
print(total_count)

94
389
93
299
27
86
68
46
1102


In [ ]:
torch.cuda.empty_cache()

In [ ]:
nlls = []
with open("outputs/debug_w_rag/log.txt") as f:
    for line in f.readlines():
        nlls.append(float(line.split()[0]))
nlls = torch.Tensor(nlls)

In [ ]:
import torch
m_ppl = torch.exp(nlls.mean()).mean()
std_ppl = torch.exp(nlls.std())

In [ ]:
print(m_ppl, std_ppl)

tensor(10.4099) tensor(20.6830)
